# sdv line

In [3]:
import os
import json
import numpy as np
from pathlib import Path
from scipy.io import loadmat
from iblatlas import atlas


def fit_line_same_depth_range(points):
    """
    Fit a straight line through the probe points using SVD
    and return points along that line spanning exactly the
    min→max DV range of the original points.
    """
    pts = np.asarray(points)
    r0 = pts.mean(axis=0)               # center
    xyz = pts - r0
    _, _, Vt = np.linalg.svd(xyz, full_matrices=False)
    direction = Vt[0]                   # dominant axis

    # Ensure DV increases (same as AP_histology logic)
    if direction[1] < 0:
        direction = -direction

    # DV range from original points
    dv_min = pts[:, 1].min()
    dv_max = pts[:, 1].max()

    # Solve for parameters t_min, t_max such that:
    #   r0 + t * direction  has DV = dv_min and dv_max
    t_min = (dv_min - r0[1]) / direction[1]
    t_max = (dv_max - r0[1]) / direction[1]

    # Generate points on this line
    t_vals = np.linspace(t_min, t_max, 200)   # 200 samples is enough for smoothness
    line_pts = r0 + np.outer(t_vals, direction)

    return line_pts


def process_single_probeccf(mat_path, atlas_obj, res=10):

    print(f"Processing: {mat_path}")

    # Load .mat
    mat = loadmat(mat_path, squeeze_me=True, struct_as_record=False)
    probe_ccf = mat["probe_ccf"]

    results_folder = mat_path.parent
    npy_paths = []

    # ---------------------------------------------------
    # Instead of saving raw points, save fitted line
    # ---------------------------------------------------
    for i in range(2):
        pts = probe_ccf[i].points
        line_pts = fit_line_same_depth_range(pts)

        npy_file = results_folder / f"probe{i+1}.npy"
        np.save(npy_file, line_pts)
        npy_paths.append(npy_file)

    # ---------------------------------------------------
    # Folder logic
    # ---------------------------------------------------
    histology_folder = mat_path.parents[1]
    animal_root = histology_folder.parent

    naive_folder = None
    recall_folder = None

    for sub in animal_root.iterdir():
        name = sub.name.lower()
        if name.endswith("_naive"):
            naive_folder = sub / "ibl"
        elif name.endswith("_recall"):
            recall_folder = sub / "ibl"

    if naive_folder is None or recall_folder is None:
        raise RuntimeError(f"Missing _naive or _recall folder in {animal_root}")

    naive_folder.mkdir(parents=True, exist_ok=True)
    recall_folder.mkdir(parents=True, exist_ok=True)

    targets = [naive_folder, recall_folder]

    # ---------------------------------------------------
    # Convert to xyz and save JSON
    # ---------------------------------------------------
    for npy_file, out_dir in zip(npy_paths, targets):

        xyz_apdvml = np.load(npy_file) * res
        xyz_mlapdv = atlas_obj.ccf2xyz(xyz_apdvml, ccf_order="apdvml") * 1e6

        xyz_picks = {"xyz_picks": xyz_mlapdv.tolist()}
        out_json = out_dir / "xyz_picks.json"

        with open(out_json, "w") as f:
            json.dump(xyz_picks, f, indent=2)

    print("Finished.")


def process_all(root_dir):
    root = Path(root_dir)
    atlas_obj = atlas.AllenAtlas(10)

    # find all probe_ccf.mat files
    probe_files = list(root.rglob("probe_ccf.mat"))

    if not probe_files:
        print("No probe_ccf.mat files found.")
        return

    for mat_path in probe_files:
        process_single_probeccf(mat_path, atlas_obj)

    print("All files processed.")
process_all(r"D:\Data\raw")


Processing: D:\Data\raw\3198-52\histology\results\probe_ccf.mat
Finished.
Processing: D:\Data\raw\3198-51\histology\results\probe_ccf.mat
Finished.
Processing: D:\Data\raw\3556-17\histology\results\probe_ccf.mat
Finished.
Processing: D:\Data\raw\7644\histology\results\probe_ccf.mat
Finished.
Processing: D:\Data\raw\7633\histology\results\probe_ccf.mat
Finished.
All files processed.


In [17]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
import scipy.io as sio

def compute_probe_angles_from_mat(mat_path):
    """
    Computes probe angles exactly like AP_histology (SVD on all points).
    Returns a list of dicts: one entry per probe.
    """
    mat = sio.loadmat(mat_path, squeeze_me=True, simplify_cells=True)
    probe_ccf = mat["probe_ccf"]

    results = []
    for probe_idx, probe in enumerate(probe_ccf, start=1):

        pts = np.asarray(probe["points"])  # Nx3, [AP, DV, ML]
        if pts.ndim != 2 or pts.shape[1] != 3:
            continue

        # -----------------------------
        # SVD: principal axis (AP_histology style)
        # -----------------------------
        r0 = pts.mean(axis=0)
        xyz = pts - r0
        _, _, Vt = np.linalg.svd(xyz, full_matrices=False)
        direction = Vt[0]  # principal direction

        # Flip so DV is positive (MATLAB: if histology_probe_direction(2) < 0)
        if direction[1] < 0:
            direction = -direction

        dx, dy, dz = direction  # (AP, DV, ML)

        # -----------------------------
        # Angles (same conventions as MATLAB plot)
        # -----------------------------
        angle_ap_dv = np.degrees(np.arctan2(dy, dx))  # sagittal tilt
        angle_ml_ap = np.degrees(np.arctan2(dz, dx))  # coronal tilt (ML vs AP)
        angle_ml_dv = np.degrees(np.arctan2(dz, dy))  # medial tilt (ML vs DV)

        results.append({
            "file": str(mat_path),
            "probe": probe_idx,
            "dir_x_ap": dx,
            "dir_y_dv": dy,
            "dir_z_ml": dz,
            "angle_ap_dv": angle_ap_dv,
            "angle_ml_ap": angle_ml_ap,
            "angle_ml_dv": angle_ml_dv
        })

    return results


def process_all_probeccf(root_folder, output_csv="probe_angles.csv"):
    """
    Walks through root_folder, finds all probe_ccf.mat, computes angles,
    saves to a single CSV.
    """
    root = Path(root_folder)
    files = list(root.rglob("probe_ccf.mat"))

    if not files:
        print("No probe_ccf.mat files found.")
        return

    all_results = []

    for mat_path in files:
        print(f"Processing {mat_path}")
        res = compute_probe_angles_from_mat(mat_path)
        all_results.extend(res)

    df = pd.DataFrame(all_results)
    df.to_csv(output_csv, index=False)

    print(f"\nSaved CSV: {output_csv}")
    print(df.head())
    return df


# -----------------------------
# RUN IT
# -----------------------------
if __name__ == "__main__":
    process_all_probeccf(r"D:\Data\raw", "D:/Data/raw/probe_angles.csv")


Processing D:\Data\raw\3198-52\histology\results\probe_ccf.mat
Processing D:\Data\raw\3198-51\histology\results\probe_ccf.mat
Processing D:\Data\raw\3556-17\histology\results\probe_ccf.mat
Processing D:\Data\raw\7644\histology\results\probe_ccf.mat
Processing D:\Data\raw\7633\histology\results\probe_ccf.mat

Saved CSV: D:/Data/raw/probe_angles.csv
                                                file  probe  dir_x_ap  \
0  D:\Data\raw\3198-52\histology\results\probe_cc...      1  0.071060   
1  D:\Data\raw\3198-52\histology\results\probe_cc...      2 -0.028752   
2  D:\Data\raw\3198-51\histology\results\probe_cc...      1  0.117155   
3  D:\Data\raw\3198-51\histology\results\probe_cc...      2  0.191832   
4  D:\Data\raw\3556-17\histology\results\probe_cc...      1  0.149150   

   dir_y_dv  dir_z_ml  angle_ap_dv  angle_ml_ap  angle_ml_dv  
0  0.069033  0.995080    44.170930    85.915371    86.031524  
1  0.830816 -0.555805    91.982067   -92.961332   -33.781998  
2  0.894208 -0.432050 